In [1]:
#fai_model = torch.nn.DataParallel(module=fai_model, device_ids=[0, 3],output_device=[0])#前向在 device_ids，梯度汇总和更新在output_device上

In [2]:

import torch
from torch.nn import DataParallel
from torch.autograd import Variable ## torch中自动计算梯度模块
import torch.nn as nn # 神经网络模块
import torch.utils.data as data
import torch.nn.functional as F  #神经网络模块中的常用功能 
import torch.multiprocessing as mp
from torch import optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader,Dataset

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision.utils import make_grid
from torchvision import datasets, models
from torchvision.models import *

from __future__ import print_function, division
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import time
import os
import copy
from tqdm import tqdm
import pandas as pd
from skimage import io, transform
from PIL import Image
%matplotlib inline
plt.ion()   # interactive mode
%config InlineBackend.figure_format = 'retina'

#显示中文字体设置
plt.rcParams["font.sans-serif"] = ["Droid Sans Fallback"]
plt.rcParams['axes.unicode_minus'] = False #为了正常显示是"-"减号

MODELS = {"resnet50":resnet50}
classes = ['collar_design_labels', 'neck_design_labels'] 
label_count = {'coat_length_labels':8,
               'collar_design_labels':5, 
               'lapel_design_labels':5,
               'neck_design_labels':5,
               'neckline_design_labels':10,
               'pant_length_labels':6, 
               'skirt_length_labels':6, 
               'sleeve_length_labels':9}
attrs_cls_label_map = {
    'skirt_length_labels':['群不可见Invisible', '短群Short', '及膝群Knee', '旗袍裙群Midi', '及脚群Ankle', '接地群Floor'],
    'coat_length_labels': ['衣不可见Invisible','高腰衣HighWaistLength','常规衣RegularLength','长衣LongLength','加长衣MicroLength',
                           '及膝衣Knee Length','旗袍衣MidiLength','及地衣Ankle&FloorLength'],
    'collar_design_labels': ['衣领不可见Invisible','衬衫领ShirtCollar','彼得潘女士小圆领PeterPan','清道夫领PuritanCollar','螺纹领RibCollar'],
    'lapel_design_labels':['翻领不可见Invisible','缺口领Notched','无领Collarless','披肩围巾式领ShawlCollar','大号披肩围巾式领PlusSizeShawl'],
    'neck_design_labels':['脖颈不可见Invisible','长高领TurtleNeck', '荷叶半高领RuffleSemi-HighCollar','低圆领LowTurtleNeck','翻领Draped Collar'],
    'neckline_design_labels':['颈领线不可见Invisible','无肩带领StraplessNeck','深V领DeepVNeckline', '直领StraightNeck', 'V领VNeckline', 
                              '方领SquareNeckline', '出肩领OffShoulder', '圆领RoundNeckline', '桃形领SweatHeartNeck', '单肩领OneShoulderNeckline'],
    'pant_length_labels':[ '裤不可见Invisible', '短裤ShortPant', '中裤Mid Length', '7分裤3/4Length', '9分裤CroppedPant', '长裤FullLength'],
    'sleeve_length_labels':['袖不可见Invisible', '无袖Sleeveless', '杯袖CupSleeves', '短袖ShortSleeves', '肘中袖ElbowSleeves',
                            '7分袖Sleeves', '及腕9分袖WristLength', '长袖LongSleeves', '超长袖ExtraLongSleeves']}


#os.environ["CUDA_VISIBLE_DEVICES"] = "0,3"
#若gpu可用则返回True
use_gpu = torch.cuda.is_available()

#保存文件后缀,即月份和日为版本尾号
#version = ""
version = '0405'
image_width = 224
epochs_num = 30
scheduler_step_size = 7 #设置含参变量的学习率变化，为多少个epoch做一次步进的衰减
learning_rate = 0.001
split_ratio = 0.2
batch_size = 64

In [3]:
#定义数据预处理
fai_data_transforms = {
                    'train': transforms.Compose([
                        transforms.Resize(image_width, interpolation=2),
                        transforms.RandomHorizontalFlip(),
                        #transforms.ColorJitter(brightness=0.3, contrast=0.2, saturation=0.2, hue=0.2),
                        #transforms.RandomRotation(15, resample=False, expand=False, center=None),
                        #transforms.RandomResizedCrop(image_width, scale=(0.95, 1.0), ratio=(0.75, 1.3333333333333333), interpolation=2),
                        
                        transforms.ToTensor(),
                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                    ]),
                    'val': transforms.Compose([
                        #transforms.Resize(256),
                        #transforms.CenterCrop(256,224),
                        transforms.Resize(image_width, interpolation=2),
                        transforms.ToTensor(),
                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                    ]),
                    'test': transforms.Compose([
                        #transforms.Resize(256),
                        #transforms.CenterCrop(256,224),
                        transforms.Resize(image_width, interpolation=2),
                        transforms.ToTensor(),
                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                    ])
}

#定义数据集
#当变换中含有 transforms.ToTensor(),处理后为RGB CHW 0-1.0数据，当含有transforms.Normalize()会返回一个分布在(x-mean)/std,这时值的范围就不是0-1.0了
def pil_loader(path):
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB').resize((224,224),Image.NEAREST)

def accimage_loader(path):
    import accimage
    try:
        return accimage.Image(path)
    except IOError:
        # Potentially a decoding problem, fall back to PIL.Image
        return pil_loader(path)

def default_loader(path):
    from torchvision import get_image_backend
    if get_image_backend() == 'accimage':
        return accimage_loader(path)
    else:
        return pil_loader(path)
#定义数据集
class FaiTrainDataset(Dataset):
    """FaiTrainDataset dataset."""

    def __init__(self, train_csv_path_and_file, train_val_images_root_dir, train =True, split_ratio=0.2,transform=None,target_transform=None,loader=default_loader):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = pd.read_csv(train_csv_path_and_file)
        #self.df_load = df.sample(frac=1).reset_index(drop=True)
        self.df_load = self.df.copy()
        #df.iloc[np.random.permutation(len(df))]
        #self.x, self.y = self.df_load.ix[:,0],self.df_load.ix[:,2:]
        #self.x_train,self.x_test,self.y_train,self.y_test = train_test_split(self.x,self.y,test_size=split_ratio,random_state=42)
        self.split_ratio = split_ratio
        self.cut_idx = int(len(self.df_load)-round(len(self.df_load)*self.split_ratio))#int(round(0.1 * self.df_load.shape[0]))
        #self.b = int(len(self.df_load)*0.2)
        self.df_train= self.df_load.iloc[:self.cut_idx]
        self.df_val = self.df_load.iloc[self.cut_idx:]
        
        self.df_train_load = self.df_train.copy()
        self.df_val_load = self.df_val.copy()
        
        self.df_train_load.columns = ['image_id', 'class', 'label']
        self.df_val_load.columns = ['image_id', 'class', 'label']
        
        self.df_train_load.reset_index(inplace= True,drop=True)
        self.df_val_load.reset_index(inplace= True,drop=True)
        
        self.train_images = self.df_train_load['image_id'].tolist()
        self.train_labels = self.df_train_load['label'].tolist()
        
        self.val_images = self.df_val_load['image_id'].tolist()
        self.val_labels = self.df_val_load['label'].tolist()
        
        n1=len(self.df_train_load)
        n2=len(self.df_val_load)
        
        #不用转化为one-hot，根据所用的交叉熵形式
        #self.train_y = np.zeros((n1, label_count[self.df_train_load['class'][0]]), dtype=np.uint8)
        #self.val_y = np.zeros((n2, label_count[self.df_val_load['class'][0]]), dtype=np.uint8)
        self.train_y = np.zeros(n1, dtype=np.uint8)
        self.val_y = np.zeros(n2, dtype=np.uint8)
        
        for i in range(n1):
            tmp_label1=self.train_labels[i]
            self.train_y[i]=tmp_label1.find('y')
        for j in range(n2):
            tmp_label2=self.val_labels[j]
            self.val_y[j]=tmp_label2.find('y')
        self.train_data = list(zip(self.train_images,self.train_y))
        self.val_data = list(zip(self.val_images,self.val_y))
        #print("训练集：batch化需要的元组样例是{0}：".format(self.train_data[0]))
        #print("验证集：batch化需要的元组样例是{0}：".format(self.val_data[0]))
        
        self.train_val_images_root_dir = train_val_images_root_dir
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader
        self.train = train

    def __len__(self):
        if self.train == True:
            return len(self.df_train_load)
        else:
            return len(self.df_val_load)

    def __getitem__(self, index): #最终返回的是Tensor
        if self.train == True:
            train_image_path, train_label = self.train_data[index]
            train_img_name = os.path.join(self.train_val_images_root_dir,train_image_path)
            #image = io.imread(img_name) #用的skimage.io,读入为uint8，RGB，HWc图像
            train_img = self.loader(train_img_name)
            
            if self.transform is not None:
                train_img = self.transform(train_img) #处理后为RGB CHW ，个位整数的数据
            if self.target_transform is not None:
                train_label = self.target_transform(train_label)
            return train_img,train_label
        else:
            val_image_path, val_label = self.val_data[index]
            val_img_name = os.path.join(self.train_val_images_root_dir,val_image_path)
            #image = io.imread(img_name) #用的skimage.io,读入为uint8，RGB，HWc图像
            val_img = self.loader(val_img_name)
            
            if self.transform is not None:
                val_img = self.transform(val_img) ##处理后为RGB CHW ，个位整数的数据
            if self.target_transform is not None:
                val_label = self.target_transform(val_label)
            return val_img,val_label #返回值是Tensor
            
        

#定义测试数据集
class FaiTestDataset(Dataset):
    """FaiTestDataset dataset."""

    def __init__(self, test_csv_path_and_file, test_images_root_dir, attr, transform=None,target_transform=None,loader=default_loader):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
       
        self.df_test = pd.read_csv(test_csv_path_and_file)
        #定义各列名称
        self.df_test.columns = ['image_id', 'class', 'x']
        del self.df_test['x']
        self.attr = attr
        
        self.df_test_load = self.df_test[(self.df_test['class'] == self.attr)].copy()
        self.df_test_load.reset_index(inplace= True,drop= True)
        
        self.test_images = self.df_test_load['image_id'].tolist()
        #n=len(self.df_test_load)
        #self.test_y = np.zeros((n, label_count[self.attr]), dtype=np.uint8)
        #self.test_data = list(zip(self.test_images,self.test_y))
        self.test_data = self.test_images
        #print("测试集（不含label）：batch化需要的元组样例是{0}：".format(self.test_data[0]))
    
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader
        self.test_images_root_dir = test_images_root_dir
    
    def __len__(self):
        return len(self.df_test_load)

    def __getitem__(self, index):
        test_path = self.test_data[index]
        test_img_name = os.path.join(self.test_images_root_dir,test_path)
        #test_image = io.imread(test_img_name) 只能用PIL读图像，因为系统要求
        test_img = self.loader(test_img_name)
            
        if self.transform is not None:
            test_img = self.transform(test_img) #处理后为RGB CHW ，个位整数的数据

        return test_img #返回值为Tensor

#测试定义的数据,一个batch的数据可视化
def fai_augment_visualize(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0)) #若用cv显示则是：inp = inp.numpy().transpose((1, 2, 0))*255 
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    #plt.pause(0.001)  # pause a bit so that plots are updated
    # opencv
    #img2 = data[i][0].numpy()*255
    #img2 = img2.astype('uint8')
    #img2 = np.transpose(img2, (1,2,0))
    #img2=img2[:,:,::-1]#RGB->BGR
    #cv2.imshow('img2', img2)
    #cv2.waitKey()

In [4]:
########################### 定义训练和测试及其他可视化函数 #######################
#定义训练模型的具体步骤及参数设置
def fai_train_model(model, criterion, optimizer, scheduler, batch_size,split_ratio,num_epochs,attr,model_key,version):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    #测试训练，验证和测试数据上的数据增强
    training_fai_train_dataset = FaiTrainDataset(train_csv_path_and_file='../train/Annotations/{0}.csv'.format(attr),
                                       train_val_images_root_dir='../train/',
                                       train =True, split_ratio=split_ratio,
                                       transform= fai_data_transforms['train'])
    training_fai_val_dataset = FaiTrainDataset(train_csv_path_and_file='../train/Annotations/{0}.csv'.format(attr),
                                       train_val_images_root_dir='../train/',
                                       train =False, split_ratio=split_ratio,
                                       transform= fai_data_transforms['val'])
    training_image_datasets = {'train': training_fai_train_dataset, 'val':training_fai_val_dataset}

    #关于训练验证集的封装
    training_dataloaders = {x: torch.utils.data.DataLoader(training_image_datasets[x], batch_size=batch_size,shuffle=True, num_workers=8)
                   for x in ['train', 'val']}
    training_dataset_lengths = {x: len(training_image_datasets[x]) for x in ['train', 'val']}
    print("摘要：Attr:{0}.Train样本数：{1} ，Val样本数：{2}".format(attr,training_dataset_lengths['train'],training_dataset_lengths['val']))
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            #每batch一次train，就val一次
            if phase == 'train':
                #与Optimizer类似的是，其主要功能体现在step()方法中，用于更新optimizer对象每个param_group字典的lr键的值。
                #scheduler = torch.optim.ReduceLROnPlateau(optimizer, 'min')，这应用在函数体外
                scheduler.step() #如scheduler.step(val_loss)
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for data in training_dataloaders[phase]:
                # get the inputs(每次一个batch)
                inputs, labels = data

                # wrap them in Variable
                if use_gpu:
                    
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.long().cuda())
                else:
                    inputs, labels = Variable(inputs,volatile=True), Variable(labels,volatile=True)
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1) #第一个是最大概率值的张量，第二个是索引值
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                # 即用的总loss而不是平均loss
                running_loss += loss.data[0] * inputs.size(0) #inputs.size(0)：每个batchsize的大小
                running_corrects += torch.sum(preds == labels.data) #可知标签不用转化为one-hot

            epoch_loss = running_loss / training_dataset_lengths[phase]
            epoch_acc = running_corrects / training_dataset_lengths[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    
    #保存最好的模型
    print("开始保存模型")
    prefix_cls = attr.split('_')[0]
    PATH = '../models/{0}/pytorch_{0}_{1}_{2}'.format(prefix_cls, model_key, version)
    torch.save(model.state_dict(), PATH)
    #model = 你定义的模型class封装
    #model.load_state_dict(torch.load(PATH))
    print("保存最好的模型完成")
    
    return model


######这是将训练好的模型用于测试集上，输出对应属性的也测结果到csv文件
#
def fai_predict(predictor,attr,model_key,version): 
    fai_test_dataset = FaiTestDataset(test_csv_path_and_file='../test/Tests/question.csv',
                                      test_images_root_dir='../test/',
                                      attr =attr,
                                      transform= fai_data_transforms['test'])
    
    test_dataloader = torch.utils.data.DataLoader(fai_test_dataset, batch_size=32,shuffle=False, num_workers=4)
    
    model = predictor
    model.train(False)
    model.eval() #model.train(False)  # Set model to evaluate mode
    #len(test_dataloader)返回的是len(相应dataset)/batch_size
    #for i_batch in tqdm(range(len(test_dataloader))):
    #在测试集上预测并保存结果
    df_test_load = fai_test_dataset.df_test_load
    print("测试属性：Attr:{0}".format(attr))
    print('测试数据集的样本数为：{0},迭代器需要的迭代次数是：{1}次batchsize的迭代'.format(len(fai_test_dataset),len(test_dataloader)))
    
    result = []
    prefix_cls = attr.split('_')[0]
    for i,batch_x in enumerate(test_dataloader):
        batch_x = Variable(batch_x, volatile=True)
        out = model(batch_x)
        out = F.softmax(out,dim=1).cpu() #把输出的正负数转到0-1之间
        test_np=out.data.numpy()
        
        #tmp_list = test_np.tolist()
        for jj in test_np:
            
            tmp_result = ''
            for tmp_ret in jj:
                tmp_result += '{:.5f};'.format(tmp_ret)
                #不要最后一个分号
            result.append(tmp_result[:-1])
    
    #预测结果导入内存表格的result列
    df_test_load['result'] = result     
    df_test_load.to_csv('../result/pytorch/pytorch_{0}_{1}_{2}.csv'.format(prefix_cls, model_key, version), header=None, index=False) 
    print('#######完成{0}:{1}下的测试机上的csv文件输出'.format(model_key,attr))
        

In [5]:
################################### 实例化训练和测试 ############################
#加载预训练模型并重写全连接层。

#state_dict = torch.utils.model_zoo.load_url('https://s3.amazonaws.com/pytorch/models/resnet18-5c106cde.pth')
#the_model = TheModelClass(*args, **kwargs)
#the_model.load_state_dict(torch.load(PATH))

#下面是冻结卷积层的方法
# fai_model = torchvision.models.resnet18(pretrained=True)
# for param in fai_model.parameters():
#     param.requires_grad = False
# Parameters of newly constructed modules have requires_grad=True by default
#model.fc = nn.Linear(512, 100)
# Optimize only the classifier
#optimizer = optim.SGD(model.fc.parameters(), lr=1e-2, momentum=0.9)
for KEY, MODLE in MODELS.items():
    print('######################在{0}下训练8个分类器####################'.format(KEY))
    print()
    for cur_class in classes:

        print('#######{0}:{1}####################'.format(KEY,cur_class ))
        #预结构
        fai_model = models.resnet18(pretrained=True)
        
        #改进的自定义添加
        num_ftrs = fai_model.fc.in_features
        fai_model.fc = nn.Linear(num_ftrs, label_count[cur_class])

        
        #打印含参数各层的名字
        #params = fai_model.state_dict()
        #for k,v in params.items():
        #    print(k)#打印网络中的变量名
        #print(fai_model)#可查最后一层Module类对象的名和输入参数，输出参数名，如(fc): Linear(in_features=512, out_features=5, bias=True)
        
        
        

        #设置采用多分类的交叉熵loss
        criterion = nn.CrossEntropyLoss()

        # 设置优化器，fai_model.parameters()表示优化全部参数@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@这里可设置两部优化法
        optimizer = optim.SGD(fai_model.parameters(), lr=learning_rate, momentum=0.9)

        # Decay LR by a factor of 0.1 every 7 epochs
        #与Optimizer类似的是，其主要功能体现在step()方法中，用于更新optimizer对象每个param_group字典的lr键的值。
        #scheduler = torch.optim.ReduceLROnPlateau(optimizer, 'min')   scheduler.step(val_loss)
        fai_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=scheduler_step_size, gamma=0.1) 
        #训练和评估
        fai_model = torch.nn.DataParallel(module=fai_model.cuda(), device_ids=[0, 3],output_device=0)#前向在 device_ids，梯度汇总和更新在output_device上
        model = fai_train_model(fai_model, criterion, optimizer, fai_lr_scheduler, batch_size=batch_size,split_ratio=split_ratio,num_epochs=epochs_num,attr=cur_class,model_key=KEY,version=version)
        #在测试集上运用训练好的模型，输出csv文件
        print('#######{0}:{1}训练完毕，开始在测试集上测试'.format(KEY,cur_class ))
        #在gpu上测试
        fai_predict(predictor = model,attr=cur_class,model_key=KEY,version=version)

######################在resnet50下训练8个分类器####################

#######resnet50:collar_design_labels####################
摘要：Attr:collar_design_labels.Train样本数：6714 ，Val样本数：1678
Epoch 0/29
----------
train Loss: 1.3834 Acc: 0.4200
val Loss: 1.1742 Acc: 0.5298

Epoch 1/29
----------
train Loss: 1.0458 Acc: 0.5977
val Loss: 1.0004 Acc: 0.6037

Epoch 2/29
----------
train Loss: 0.8753 Acc: 0.6631
val Loss: 0.8816 Acc: 0.6657

Epoch 3/29
----------
train Loss: 0.7353 Acc: 0.7230
val Loss: 0.8349 Acc: 0.6853

Epoch 4/29
----------
train Loss: 0.6144 Acc: 0.7800
val Loss: 0.7958 Acc: 0.7056

Epoch 5/29
----------
train Loss: 0.5112 Acc: 0.8187
val Loss: 0.7701 Acc: 0.7151

Epoch 6/29
----------
train Loss: 0.4240 Acc: 0.8558
val Loss: 0.8008 Acc: 0.7116

Epoch 7/29
----------
train Loss: 0.3379 Acc: 0.8984
val Loss: 0.7541 Acc: 0.7300

Epoch 8/29
----------
train Loss: 0.3173 Acc: 0.9048
val Loss: 0.7594 Acc: 0.7253

Epoch 9/29
----------
train Loss: 0.3158 Acc: 0.9005
val Loss: 0.7548 Acc: 0.72